In [2]:
import requests
import pandas as pd
import json
import re
from pandas import concat 

In [17]:
    url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin_address}&destinations=Istanbul&key={api_key}'
    data = check(url)
    data

{'destination_addresses': ['İstanbul, Türkiye'],
 'origin_addresses': ['Tel Aviv-Yafo, Israel'],
 'rows': [{'elements': [{'distance': {'text': '1,815 km', 'value': 1815227},
     'duration': {'text': '21 hours 3 mins', 'value': 75772},
     'status': 'OK'}]}],
 'status': 'OK'}

In [18]:
api_key = ''
origin_address = 'Tel Aviv'

df = pd.DataFrame(columns=['Target', 'Distance_km', 'Duration (hour+minutes)', 'Longitude', 'latitude'])
#____________________________________________________________________________________________________
def check(url):
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return response_data
#____________________________________________________________________________________________________
def get_distance_time(destination_address):
    # Build URL
    url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin_address}&destinations={destination_address}&key={api_key}'
    data = check(url)
    # distance
    distance = data["rows"][0]["elements"][0]['distance']['text']
    distance = re.findall('([0-9]+,[0-9]+) km',distance)[0]
    distance = float(distance.replace(",", ""))

    # time
    time = data["rows"][0]["elements"][0]['duration']['text']
    days = re.findall('([0-9]+) day',time)
    hours = re.findall('([0-9]+) hours',time) 
    minutes = re.findall('([0-9]+) mins',time)
    seconds = re.findall('([0-9]+) sec',time)
    if not days:
        days = [0]
    if not hours:
        hours = [0]
    if not minutes:
        minutes = [0]
    if not seconds:
        seconds = [0]
    total_hours = int(days[0])*24+int(hours[0])
    total_minutes = int(int(minutes[0])+(int(seconds[0])/60))
    total_time = f"{total_hours} hours + {total_minutes} minutes"
    return distance,total_time
#____________________________________________________________________________________________________
def get_lat_lng(destination_address):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination_address,api_key)
    lat = 0
    lng = 0
    response_data = check(url)

    lat = response_data['results'][0]["geometry"]["location"]["lat"]
    lng = response_data['results'][0]["geometry"]["location"]["lng"]
    return lat,lng
    
file =  open("C:/Users/liron/OneDrive - Ariel University/Toar 1/third year/Yeda&Netunim/Jupyter/HW3/dests.txt")
for city in file:
    destination_address = city.strip()
    distance,total_time = get_distance_time(destination_address)
    lat,lng = get_lat_lng(destination_address)
    new_row = pd.Series([destination_address, distance, total_time,lng, lat], index=df.columns)
    df = concat([df, new_row.to_frame().transpose()], ignore_index=True)

In [19]:
df

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
0,Istanbul,1815.0,21 hours + 3 minutes,28.978359,41.008238
1,Amsterdam,4527.0,48 hours + 0 minutes,4.904139,52.367573
2,Valletta,3793.0,51 hours + 0 minutes,14.5141,35.899237
3,Basel,4093.0,44 hours + 0 minutes,7.588576,47.559599
4,Doha,2164.0,22 hours + 39 minutes,51.53104,25.285447


In [20]:
df['Distance_km'] = df['Distance_km'].astype(float)
top_three = df['Distance_km'].nlargest(3).index
top_three = df.loc[top_three]
top_three

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
1,Amsterdam,4527.0,48 hours + 0 minutes,4.904139,52.367573
3,Basel,4093.0,44 hours + 0 minutes,7.588576,47.559599
2,Valletta,3793.0,51 hours + 0 minutes,14.5141,35.899237
